In [76]:
import pandas as pd
import numpy as np
from glob import glob
import json

In [77]:
text = {
    'G': pd.read_csv('./gpt_text_eval.csv').fillna(0),
    'L': pd.read_csv('./llama_text_eval.csv').fillna(0),
    'GT': pd.read_csv('./gpt_text2sql_text_eval.csv').fillna(0),
    'GR': pd.read_csv('./gpt_rag_text_eval.csv').fillna(0),
    'LT': pd.read_csv('llama_text2sql_text_eval.csv').fillna(0),
    'LR': pd.read_csv('llama_rag_text_eval.csv').fillna(0),
    'R': pd.read_csv('shuffled_text_eval.csv').fillna(0),
}

In [78]:
output_types = ['count', 'name', 'area', 'length', 'loc', 'angle', 'distance']

In [79]:
#text['G'].groupby('type')[['P','R','F1']].mean()

text_agg = {k: text[k].groupby('type')[['P','R','F1']].mean().add_suffix('_' + k) for k in text}

merged_df = pd.merge(text_agg['G'], text_agg['L'], left_index=True, right_index=True)
merged_df = pd.merge(merged_df, text_agg['GT'], left_index=True, right_index=True)
merged_df = pd.merge(merged_df, text_agg['LT'], left_index=True, right_index=True)
merged_df = pd.merge(merged_df, text_agg['GR'], left_index=True, right_index=True)
merged_df = pd.merge(merged_df, text_agg['LR'], left_index=True, right_index=True)
merged_df = pd.merge(merged_df, text_agg['R'], left_index=True, right_index=True)
merged_df = merged_df[sorted(merged_df.columns)]

In [80]:
types_ordered = ['range+name', 'range:non_spat_filter+name', 'range:direction+name', 'range:towards+name', 
 'knn+name', 'knn:non_spat_filter+name', 'knn+name+multihop1', 'knn+name+multihop2',
 'knn:direction+name', 'knn:towards+name', 'intersects:area_max+name', 'intersects:length_max+name',
 'range+loc', 'range:non_spat_filter+loc','range:direction+loc',  'range:towards+loc',
  'knn+loc', 'knn:non_spat_filter+loc', 'knn:direction+loc',  'knn:towards+loc', 
  'range+angle', 'knn+angle',
  'range+count', 'intersects+count',
'range+distance', 'knn+distance', 
'intersects:area_total+area', 'intersects:length_total+length'
 ]
print(len(types_ordered))
type_labels = {types_ordered[i-1]: 'T%d' % i for i in range(1,29)}

type_labels

28


{'range+name': 'T1',
 'range:non_spat_filter+name': 'T2',
 'range:direction+name': 'T3',
 'range:towards+name': 'T4',
 'knn+name': 'T5',
 'knn:non_spat_filter+name': 'T6',
 'knn+name+multihop1': 'T7',
 'knn+name+multihop2': 'T8',
 'knn:direction+name': 'T9',
 'knn:towards+name': 'T10',
 'intersects:area_max+name': 'T11',
 'intersects:length_max+name': 'T12',
 'range+loc': 'T13',
 'range:non_spat_filter+loc': 'T14',
 'range:direction+loc': 'T15',
 'range:towards+loc': 'T16',
 'knn+loc': 'T17',
 'knn:non_spat_filter+loc': 'T18',
 'knn:direction+loc': 'T19',
 'knn:towards+loc': 'T20',
 'range+angle': 'T21',
 'knn+angle': 'T22',
 'range+count': 'T23',
 'intersects+count': 'T24',
 'range+distance': 'T25',
 'knn+distance': 'T26',
 'intersects:area_total+area': 'T27',
 'intersects:length_total+length': 'T28'}

In [81]:
text_by_output = {o: merged_df[merged_df.index.str.contains(r'\+%s' % o)]  for o in output_types}

In [82]:
parsed = {
    'G': pd.read_csv('./gpt_parsed_eval.csv').fillna(0),
    'L': pd.read_csv('./llama_parsed_eval.csv').fillna(0),
    'GT': pd.read_csv('./gpt_text2sql_parsed_eval.csv').fillna(0),
    'LT': pd.read_csv('./llama_text2sql_parsed_eval.csv').fillna(0),
    'GR': pd.read_csv('./gpt_rag_parsed_eval.csv').fillna(0),
    'LR': pd.read_csv('./llama_rag_parsed_eval.csv').fillna(0),
    'R': pd.read_csv('./shuffled_parsed_eval.csv').fillna(0),
}
cols = ['P','R','F1', 'distance_error', 'angle_error', 'relative_error']
relevant_scores = {
    'count': ['relative_error'],
    'area': ['relative_error'],
    'length': ['relative_error'],
    'distance': ['relative_error'],
    'name': ['P','R', 'F1'],
    'loc': ['P','R', 'F1', 'distance_error'],
    'angle': ['P','R', 'F1','angle_error']
}

def get_output_type(t):
    if 'name' in t or 'multihop' in t:
        return 'name'
    elif 'count' in t:
        return 'count'
    elif 'area' in t:
        return 'area'
    elif 'length' in t:
        return 'length'
    elif 'distance' in t:
        return 'distance'
    elif 'angle' in t:
        return 'angle'
    elif 'loc' in t:
        return 'loc'
    
def get_score(t):
    print(t)
    scores = relevant_scores[t]
    return [s for s in scores if 'P' != s and 'R' != s]

In [83]:
for k in parsed:
    parsed[k]['output_type'] = parsed[k]['type'].map(lambda t: get_output_type(t))
    parsed[k]['relative_error'] = parsed[k]['relative_error'].clip(upper=1.0)
summary_table = {k: parsed[k].groupby('output_type')['attempted'].mean() for k in parsed}
# summary_table2 = {k: parsed[k].groupby('output_type')[get_score(get_output_type(k))+['attempted']].mean() for k in parsed}
attempted_summary = pd.DataFrame(summary_table)
attempted_summary

,G,L,GT,LT,GR,LR,R
output_type,,,,,,,
angle,0.910000,0.920000,0.8050,0.845000,0.870000,0.725000,0.910000
area,0.990000,0.990000,0.7600,0.990000,0.890000,0.440000,1.000000
count,0.930000,0.945000,0.9000,0.865000,0.870000,0.745000,0.530000
distance,0.990000,1.000000,0.9600,0.965000,0.935000,0.800000,0.970000
length,0.490000,0.830000,0.5600,0.800000,0.310000,0.340000,0.850000
loc,0.997500,0.990000,0.9700,0.892500,0.743750,0.950000,0.975000
name,0.973333,0.954167,0.9275,0.924167,0.771667,0.820833,0.961667


In [84]:
from collections import defaultdict
summary_lists = defaultdict(list)
scores = {'F1': ['name', 'loc', 'angle'], 'distance_error': ['loc'], 'angle_error': ['angle'], 'relative_error': ['count', 'distance', 'length', 'area']}
for k in parsed:
    for s in scores:
        for o in relevant_scores:
            if k == 'G' and s == 'F1':
                summary_lists['output_type'].append(o)
            if o in scores[s]:
                summary_lists[s + '_' + k].append(parsed[k][(parsed[k]['output_type'] == o)][s].mean())
            else:
                summary_lists[s + '_' + k].append(None)
            

In [85]:
df1 = pd.DataFrame(summary_lists)
df1

,output_type,F1_G,distance_error_G,angle_error_G,relative_error_G,F1_L,distance_error_L,angle_error_L,relative_error_L,F1_GT,...,angle_error_GR,relative_error_GR,F1_LR,distance_error_LR,angle_error_LR,relative_error_LR,F1_R,distance_error_R,angle_error_R,relative_error_R
0,count,NaN,NaN,NaN,0.882197,NaN,NaN,NaN,0.795867,NaN,...,NaN,0.982284,NaN,NaN,NaN,0.891879,NaN,NaN,NaN,0.982364
1,area,NaN,NaN,NaN,0.986023,NaN,NaN,NaN,0.919722,NaN,...,NaN,0.993371,NaN,NaN,NaN,0.995512,NaN,NaN,NaN,0.979083
2,length,NaN,NaN,NaN,0.925792,NaN,NaN,NaN,0.895768,NaN,...,NaN,0.948860,NaN,NaN,NaN,0.955735,NaN,NaN,NaN,1.000000
3,distance,NaN,NaN,NaN,0.916330,NaN,NaN,NaN,0.953054,NaN,...,NaN,0.929001,NaN,NaN,NaN,0.949397,NaN,NaN,NaN,0.904186
4,name,0.232057,NaN,NaN,NaN,0.197664,NaN,NaN,NaN,0.272235,...,NaN,NaN,0.140083,NaN,NaN,NaN,0.191850,NaN,NaN,NaN
5,loc,0.206208,0.420529,NaN,NaN,0.163814,0.586533,NaN,NaN,0.214835,...,NaN,NaN,0.151601,0.654204,NaN,NaN,0.200098,0.787466,NaN,NaN
6,angle,0.345000,NaN,0.406173,NaN,0.315000,NaN,0.403799,NaN,0.310000,...,0.440248,NaN,0.210000,NaN,0.546227,NaN,0.295000,NaN,0.416514,NaN


In [86]:
df1 = pd.DataFrame(summary_lists)
for k in parsed:
    df1['error_' + k] = df1[[c for c in df1.columns if 'error' in c and c.endswith('_'+k)]].fillna(0).sum(axis=1)
df1 = df1[[c for c in df1.columns if 'relative_error' not in c and 'angle_error' not in c and 'distance_error' not in c]]
df1 = df1[sorted(df1.columns)]
summary_df = pd.merge(attempted_summary.add_prefix('attempted_').reset_index(), df1).sort_values(by='output_type', key=lambda series: series.apply(lambda x: {'name': 0, 'loc': 1, 'angle': 2}.get(x, 3)))
summary_df = summary_df[['output_type'] + sorted(c for c in summary_df.columns if 'attempted' in c and c != 'output_type') + sorted(c for c in summary_df.columns if 'attempted' not in c and c != 'output_type')]
summary_df.round(2).to_csv('./exp_tables/summary_table.csv',index=False)

summary_df[[c for c in summary_df.columns if 'error' in c]]

,error_G,error_GR,error_GT,error_L,error_LR,error_LT,error_R
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.420529,0.677745,0.437719,0.586533,0.654204,0.643140,0.787466
0,0.406173,0.440248,0.478337,0.403799,0.546227,0.458029,0.416514
1,0.986023,0.993371,0.993685,0.919722,0.995512,0.918338,0.979083
2,0.882197,0.982284,0.886060,0.795867,0.891879,0.833019,0.982364
3,0.916330,0.929001,0.957064,0.953054,0.949397,0.948838,0.904186
4,0.925792,0.948860,0.901854,0.895768,0.955735,0.922320,1.000000


In [87]:
for b in parsed:
    parsed[b] = parsed[b][parsed[b]['attempted'] == True]

In [88]:
parsed_by_output = {}
for o in relevant_scores:
    parsed_agg = {k: parsed[k][parsed[k]['type'].str.contains(r'\+%s' % o)].groupby('type')[relevant_scores[o]].mean().add_suffix('_' + k) for k in parsed}
    parsed_df = pd.merge(parsed_agg['G'], parsed_agg['L'], left_index=True, right_index=True)
    parsed_df = pd.merge(parsed_df, parsed_agg['GT'], left_index=True, right_index=True)
    parsed_df = pd.merge(parsed_df, parsed_agg['LT'], left_index=True, right_index=True)
    parsed_df = pd.merge(parsed_df, parsed_agg['GR'], left_index=True, right_index=True)
    parsed_df = pd.merge(parsed_df, parsed_agg['LR'], left_index=True, right_index=True)
    parsed_df = pd.merge(parsed_df, parsed_agg['R'], left_index=True, right_index=True)
    parsed_by_output[o] = parsed_df[sorted(parsed_df.columns)]


In [89]:
# answers_files
answers = {
    'G': {
        'text': './answers_gpt.json',
        'json': './json_answers_gpt.json'
    },
    'GT': {
        'text': './text2sql_answers_gpt.json',
        'json': './text2sql_json_answers_gpt.json'
    }, 
    'GR': {
        'text': './rag_answers_gpt.json',
        'json': './rag_json_answers_gpt.json'
    },   
    'L': {
        'text': './answers_llama.json',
        'json': './json_answers_llama.json'
    },
    'LT': {
        'text': './text2sql_answers_llama.json',
        'json': './text2sql_json_answers_llama.json'
    },
    'LR': {
        'text': './rag_answers_llama.json',
        'json': './rag_json_answers_llama.json'
    }
}

for b in answers:
    for t in answers[b]:
        with open(answers[b][t], 'r') as file:
            answers[b][t] = {a['id']: a['content'] for a in json.loads(file.read())}

In [90]:
scores = {}
for b in text:
    scores[b] = {'text': {}, 'json': {}}
    for i, row in text[b].iterrows():
        d = row.to_dict()
        scores[b]['text'][row['id']] = {k: d[k] for k in d if k not in ['type', 'id']}

for b in parsed:
    for i, row in parsed[b].iterrows():
        d = row.to_dict()
        rs = []
        for o in relevant_scores:
            if d['type'].endswith(o):
                rs = relevant_scores[o]
                break
        scores[b]['json'][row['id']] = {k: d[k] for k in d if (k not in ['type', 'id'] and k in rs)}

In [91]:
# sorting_scores = ['relative_error', 'distance_error', 'angle_error', 'F1']
# template_files = glob('./selected_questions/*.jsonl')

# for baseline in answers:
#     for path in template_files:
#         questions = []
#         question_type = path[path.rfind('/')+1:-6]
#         with open(path, 'r') as file:
#             for i in range(100):
#                 line = file.readline()
#                 question = json.loads(line)
#                 question['type'] = question_type
#                 questions.append(question)
#             collection = []
#             for q in questions:
#                 s  = str(q['id']) + '\n'
#                 s += q['question'] + '\n'
#                 s += 'true_answer: ' + json.dumps([{k: a[k] for k in a if k != 'geometry'} for a in q['answers']]) + '\n\n'
#                 s += 'text: ' + answers[baseline]['text'][q['id']] + '\n'
#                 s += 'text scores: ' + json.dumps(scores[baseline]['text'][q['id']]) + '\n'
#                 s += 'json: ' + answers[baseline]['json'][q['id']] + '\n'
#                 s += 'json scores: ' + json.dumps(scores[baseline]['json'][q['id']]) + '\n\n\n\n'
#                 _c = 0.0
#                 for c in sorting_scores:
#                     if c in scores[baseline]['json'][q['id']]:
#                         _c = scores[baseline]['json'][q['id']][c]
#                         break
#                 collection.append((_c, s))
#             collection = sorted(collection, key=lambda x: x[0])
#             with open('./printed_answers/%s_%s.txt' % (question_type, baseline), 'w') as file:
#                 for (_, t) in collection:
#                     file.write(t)

            
# # for each baseline
# # print question,answers, and all scores into a file


In [92]:
# recall table for full text evaluation

key = 'name'
df = text_by_output[key][[c for c in text_by_output[key] if 'R_' in c]]
df = df.rename(columns={c: c[c.rfind('_')+1:] for c in df.columns})
df = df.reset_index()
df['type'] = df['type'].map(lambda x: type_labels[x])
df = df.sort_values(by='type', key=lambda arr: [int(x[1:]) for x in arr])
df.round(4).to_csv('./exp_tables/name_full_text_recall.csv', index=False)
df1 = df

In [93]:
# F1 table for entity name
key = 'name'
df = parsed_by_output['name'][[c for c in text_by_output[key] if 'F1_' in c]]
df = df.rename(columns={c: c[c.rfind('_')+1:] for c in df.columns})
df = df.reset_index()
df['type'] = df['type'].map(lambda x: type_labels[x])
df = df.sort_values(by='type', key=lambda arr: [int(x[1:]) for x in arr])
df.round(4).to_csv('./exp_tables/name_parsed_f1.csv', index=False)
df2 = df

In [94]:
# entity name evaluation table
df = pd.merge(df1, df2, on='type')
df.round(4).to_csv('./exp_tables/name_full_recall_parsed_f1.csv', index=False)
df

,type,G_x,GR_x,GT_x,L_x,LR_x,LT_x,R_x,G_y,GR_y,GT_y,L_y,LR_y,LT_y,R_y
0,T1,0.074505,0.030522,0.061633,0.081202,0.066085,0.076469,0.000000,0.391330,0.430423,0.362280,0.347496,0.289249,0.340745,0.348837
1,T2,0.093572,0.017217,0.128590,0.129786,0.124809,0.119288,0.000000,0.262072,0.232191,0.385587,0.269201,0.242972,0.263367,0.291552
2,T3,0.107868,0.058146,0.137509,0.100125,0.078428,0.104713,0.000000,0.302961,0.217122,0.345080,0.289701,0.219161,0.256712,0.298708
3,T4,0.028344,0.018340,0.021316,0.036821,0.025012,0.028451,0.000000,0.477127,0.447519,0.398375,0.464108,0.360075,0.409975,0.470208
4,T5,0.183762,0.056762,0.424595,0.175095,0.133262,0.155929,0.118333,0.171589,0.084144,0.418968,0.099183,0.111253,0.090239,0.117333
5,T6,0.137167,0.088929,0.459571,0.206857,0.142024,0.197690,0.145762,0.136917,0.136587,0.449309,0.161678,0.122812,0.145821,0.149738
6,T7,0.014579,0.019833,0.064119,0.210909,0.138218,0.229000,0.000000,0.010141,0.017683,0.011416,0.026202,0.010909,0.043739,0.039448
7,T8,0.154000,0.083167,0.147333,0.166000,0.148500,0.165500,0.104000,0.146328,0.105996,0.143246,0.119236,0.110032,0.111161,0.097239
8,T9,0.151333,0.086833,0.181667,0.162333,0.120333,0.160667,0.000000,0.129762,0.082492,0.168967,0.121373,0.069143,0.115128,0.133786
9,T10,0.120333,0.048333,0.092000,0.121167,0.090833,0.099500,0.000000,0.087617,0.044068,0.084354,0.105826,0.068401,0.083343,0.086040


In [95]:
key = 'loc'
df = parsed_by_output[key]
df = df.reset_index()
df['type'] = df['type'].map(lambda x: type_labels[x])
df = df.sort_values(by='type', key=lambda arr: [int(x[1:]) for x in arr])
# df.round(2).to_csv('./exp_tables/name_parsed_f1.csv', index=False)
df = df[[c for c in df.columns if 'R_' not in c and 'P_' not in c]]
df = df.rename(columns={c: c[c.rfind('_')+1:] for c in df.columns})
df.round(4).to_csv('./exp_tables/loc_parsed_addr_f1_dist_error.csv', index=False)
df

,type,G,GR,GT,L,LR,LT,R,G,GR,GT,L,LR,LT,R
4,T13,0.272799,0.193358,0.251335,0.243726,0.215031,0.236366,0.259016,0.449493,0.533326,0.482655,0.535953,0.576638,0.515879,0.842340
6,T14,0.235154,0.198373,0.253242,0.186008,0.172995,0.210372,0.237022,0.340583,0.544118,0.446338,0.553520,0.631753,0.536667,0.801918
5,T15,0.201234,0.172691,0.199981,0.144391,0.127268,0.176084,0.203349,0.449202,0.517575,0.535630,0.544393,0.601769,0.520996,0.655307
7,T16,0.381413,0.345923,0.375420,0.258834,0.280333,0.275970,0.360212,0.185435,0.388886,0.192938,0.495335,0.684942,0.625356,0.684051
0,T17,0.191798,0.149333,0.242428,0.161806,0.161736,0.175740,0.078645,0.460883,0.574156,0.393751,0.695264,0.578485,0.617841,0.869191
2,T18,0.102741,0.093008,0.173509,0.104573,0.090930,0.071005,0.162541,0.638131,0.635908,0.541949,0.605076,0.672741,0.660487,0.798422
1,T19,0.141804,0.100436,0.159884,0.090172,0.108973,0.095810,0.145314,0.490626,0.567005,0.435984,0.613483,0.566275,0.634148,0.807822
3,T20,0.125390,0.064566,0.133292,0.138216,0.110673,0.159322,0.199247,0.338164,0.739110,0.317956,0.611004,0.766266,0.707615,0.800329


In [96]:
key = 'angle'
df = parsed_by_output[key]
df = df.reset_index()
df['type'] = df['type'].map(lambda x: type_labels[x])
df = df.sort_values(by='type', key=lambda arr: [int(x[1:]) for x in arr])
df = df[[c for c in df.columns if 'R_' not in c and 'P_' not in c]]
df = df.rename(columns={c: c[c.rfind('_')+1:] for c in df.columns})
df.round(4).to_csv('./exp_tables/angle_parsed_dir_angle_error.csv', index=False)
df

,type,G,GR,GT,L,LR,LT,R,G,GR,GT,L,LR,LT,R
1,T21,0.623529,0.542169,0.620253,0.569892,0.445946,0.558140,0.531250,0.164834,0.226565,0.194703,0.199265,0.262617,0.206526,0.196374
0,T22,0.164948,0.186813,0.158537,0.109890,0.126761,0.180723,0.093023,0.507461,0.475218,0.503486,0.508001,0.490307,0.516201,0.540127


In [97]:
keys = ['count', 'length', 'distance', 'area']
df = parsed_by_output[keys[0]]
df = df.reset_index()
for k in keys[1:]:
    df = pd.concat([df, parsed_by_output[k].reset_index()], ignore_index=True)

df['TID'] = df['type'].map(lambda x: type_labels[x])
df = df.sort_values(by='TID', key=lambda arr: [int(x[1:]) for x in arr])
# df = df[[c for c in df.columns if 'R_' not in c and 'P_' not in c]]
df = df.rename(columns={c: c[c.rfind('_')+1:] for c in df.columns})
df = df[['type', 'TID', 'G', 'GR', 'GT', 'L', 'LR', 'LT', 'R']]
df.round(4).to_csv('./exp_tables/numeric_relative_error.csv', index=False)
df

,type,TID,G,GR,GT,L,LR,LT,R
1,range+count,T23,0.773046,0.999560,0.777839,0.742868,0.781786,0.754459,0.961913
0,intersects+count,T24,0.978022,0.953947,0.971108,0.827339,0.917906,0.856508,0.976080
4,range+distance,T25,0.949315,0.928021,0.957143,0.958014,0.940138,0.962958,0.808371
3,knn+distance,T26,0.880965,0.920472,0.953483,0.948094,0.932494,0.930840,1.000000
5,intersects:area_total+area,T27,0.985882,0.992552,0.991691,0.918911,0.989801,0.917513,0.979083
2,intersects:length_total+length,T28,0.848555,0.835034,0.824738,0.874420,0.869807,0.902900,1.000000


In [98]:
import json
sql_llama = ''
with open('./sql_outputs_llama.json', 'r') as file:
    sql_llama = json.loads(file.read())


In [99]:
sql_gpt = ''
with open('./sql_outputs_gpt.json', 'r') as file:
    sql_gpt = json.loads(file.read())


In [100]:
sql_llama_eval = {'valid_sql': 0, 'invalid_sql': 0, 'timeout': 0, 'syntax_error': 0,
                  'function_does_not_exist': 0, 'operator_does_not_exist': 0, 'column_does_not_exist': 0,
                  'relation_does_not_exist': 0, 'missing_from': 0, 'sub_query_error': 0, 'other': 0, 'sql_errors': []}
for r in sql_llama:
    _r = r['records']
    error_keys = []
    for r in _r:
        if len(r['error']):
            if 'canceling statement due to statement timeout' in r['error']:
                error_keys.append('timeout')
            elif 'syntax error' in r['error'] or 'invalid input syntax' in r['error'] or 'parse error' in r['error']:
                error_keys.append('syntax_error')
            elif 'function' in r['error'] and 'does not exist' in r['error']:
                error_keys.append('function_does_not_exist')
            elif 'operator' in r['error'] and 'does not exist' in r['error']:
                error_keys.append('operator_does_not_exist')
            elif 'column' in r['error'] and ('does not exist' in r['error'] or 'is ambiguous' in r['error']):
                error_keys.append('column_does_not_exist')
            elif ('relation' in r['error'] or 'schema' in r['error']) and 'does not exist' in r['error']:
                error_keys.append('relation_does_not_exist')
            elif 'missing FROM-clause entry' in r['error']:
                error_keys.append('missing_from')
            elif 'more than one row returned by a subquery used as an expression' in r['error']:
                error_keys.append('sub_query_error')
            else:
                error_keys.append('other')
                sql_llama_eval['sql_errors'].append(r['error'])
        else:
            error_keys.append('valid')
    print(error_keys)
    if 'valid' in error_keys or 'timeout' in error_keys:
        sql_llama_eval['valid_sql'] += 1
    else:
        sql_llama_eval['invalid_sql'] += 1
    if 'valid' not in error_keys:
        if len(error_keys) > 1:
            print(error_keys)
        sql_llama_eval[error_keys[0]] += 1


['timeout']
['function_does_not_exist']
['syntax_error']
['operator_does_not_exist']
['column_does_not_exist']
['timeout']
['relation_does_not_exist']
['timeout']
[]


IndexError: list index out of range

In [ ]:
sql_gpt_eval = {'valid_sql': 0, 'invalid_sql': 0, 'timeout': 0, 'syntax_error': 0,
                  'function_does_not_exist': 0, 'operator_does_not_exist': 0, 'column_does_not_exist': 0,
                  'relation_does_not_exist': 0, 'missing_from': 0, 'sub_query_error': 0, 'other': 0, 'sql_errors': []}
for r in sql_gpt:
    _r = r['records']
    for r in _r:
        if len(r['error']):
            is_valid = False
            if 'canceling statement due to statement timeout' in r['error']:
                error_key = 'timeout'
                is_valid = True
            elif 'syntax error' in r['error'] or 'invalid input syntax' in r['error'] or 'parse error' in r['error']:
                error_key = 'syntax_error'
            elif 'function' in r['error'] and 'does not exist' in r['error']:
                error_key = 'function_does_not_exist'
            elif 'operator' in r['error'] and 'does not exist' in r['error']:
                error_key = 'operator_does_not_exist'
            elif 'column' in r['error'] and ('does not exist' in r['error'] or 'is ambiguous' in r['error']):
                error_key = 'column_does_not_exist'
            elif ('relation' in r['error'] or 'schema' in r['error']) and 'does not exist' in r['error']:
                error_key = 'relation_does_not_exist'
            elif 'missing FROM-clause entry' in r['error']:
                error_key = 'missing_from'
            elif 'more than one row returned by a subquery used as an expression' in r['error']:
                error_key = 'sub_query_error'
            else:
                error_key = 'other'
                sql_gpt_eval['sql_errors'].append(r['error'])
    if is_valid:
        sql_gpt_eval['valid_sql'] += 1
    else:
        sql_gpt_eval['invalid_sql'] += 1
    if error_key != '':
        sql_gpt_eval[error_key] += 1


In [ ]:
import pandas as pd
df_dict = {'Name': [], 'L': [], 'G': []}
for k in sql_gpt_eval:
    if 'sql_errors' in k:
        continue
    else:
        df_dict['Name'].append(k)
        df_dict['L'].append(sql_llama_eval[k])
        df_dict['G'].append(sql_gpt_eval[k])

df = pd.DataFrame(df_dict)
df.to_csv('./exp_tables/text2sql_errors_summary.csv', index=True)
df

,Name,L,G
0,valid_sql,636,391
1,invalid_sql,2164,2409
2,timeout,521,391
3,syntax_error,731,74
4,function_does_not_exist,681,727
5,operator_does_not_exist,115,4
6,column_does_not_exist,298,1316
7,relation_does_not_exist,169,0
8,missing_from,125,10
9,sub_query_error,4,197
